The input is the .txt output from the simulation in https://jury-deliberation-app-prp3y5oy5a-zf.a.run.app/

ADD THE FILE PATH

In [ ]:
# ―― path to your log file ―――――――――――――――――――――――――――――――
file_path = "/content/[STARTED] Starting 1 deliberation r.txt"

In [ ]:
"""
Parses a jury‑simulation log file and returns a list—one item per run—where each item is a dictionary mapping every juror to their chronological ["comment", "stance"] pairs.
 ** The final comment and stance are the final verdict of the juror in a specific run !! **

Shape of the output:
[
  {                       # Run 1
    "Juror A": [["…", "UNDECIDED"], ["…", "GUILTY"]],
    "Juror B": [["…", "UNDECIDED"], ["…", "NOT GUILTY"]],
    …
  },
  { … },                  # Run 2
  …
]
"""
import re
from pathlib import Path
from typing import List, Dict


# ───────────────────────────────────────────────────────────

# regex helpers
RUN_HDR        = re.compile(r"^=== Run \d+/\d+")
DELIB_BEGIN    = "=== JURY DELIBERATION BEGINS"
COLLECT_FINAL  = "=== COLLECTING FINAL VERDICTS"
FINAL_HDR      = "=== FINAL VERDICTS"
COMMENT_RE     = re.compile(r"^\s*([^:\n]+):\s*(.*)")
STANCE_RE      = re.compile(r"^\[Current stance:\s*([A-Z ]+)\]\s*$")
EXCLUDE_NAMES  = {
    "moderator", "final verdict", "final_verdict",
    "final tally", "jury decision"
}

# ―― helpers ――――――――――――――――――――――――――――――――――――――――――――
def _split_runs(lines: List[str]) -> List[List[str]]:
    idxs = [i for i, ln in enumerate(lines) if RUN_HDR.match(ln)]
    return [lines] if not idxs else [
        lines[s:e] for s, e in zip(idxs, idxs[1:] + [len(lines)])
    ]

def _parse_single_run(chunk: List[str]) -> Dict[str, List[List[str]]]:
    jurors: Dict[str, List[List[str]]] = {}

    # boundaries of deliberation
    try:
        start = next(i for i, ln in enumerate(chunk) if DELIB_BEGIN in ln) + 1
    except StopIteration:
        return jurors
    end = next(
        (i for i, ln in enumerate(chunk[start:], start) if COLLECT_FINAL in ln),
        len(chunk),
    )

    # deliberation rounds
    i = start
    while i < end:
        ln = chunk[i]
        if ln.lstrip().startswith("["):                 # skip stance‑only lines
            i += 1; continue
        if (m := COMMENT_RE.match(ln)):
            name, text = m.group(1).strip(), m.group(2).strip()
            if name.lower() in EXCLUDE_NAMES or name.startswith("["):
                i += 1; continue
            # look‑ahead for stance
            stance = None
            j = i + 1
            while j < end:
                if (sm := STANCE_RE.match(chunk[j])):
                    stance = sm.group(1).strip(); break
                if COMMENT_RE.match(chunk[j]) or "Moderator:" in chunk[j]:
                    break
                j += 1
            jurors.setdefault(name, []).append([text, stance])
        i += 1

    # final verdicts
    try:
        fv_start = next(i for i, ln in enumerate(chunk) if FINAL_HDR in ln) + 1
    except StopIteration:
        fv_start = None

    if fv_start is not None:
        for ln in chunk[fv_start:]:
            if not ln.strip():
                continue
            m = COMMENT_RE.match(ln)
            if not m:
                break
            name, rest = m.group(1).strip(), m.group(2).strip()
            if name.lower() in EXCLUDE_NAMES:
                continue
            stance_m = re.search(r"\b(GUILTY|NOT GUILTY|UNDECIDED)\b",
                                 rest, re.I)
            if not stance_m:
                continue
            stance = stance_m.group(1).upper()
            comment = rest[stance_m.end():].lstrip(":- ").strip()
            comment = re.sub(r"^VERDICT:\s*", "", comment, flags=re.I)
            jurors.setdefault(name, []).append([comment, stance])

    return jurors

def parse_jury_log(path: str | Path) -> List[Dict[str, List[List[str]]]]:
    """
    Parse one log file and return a *list*.
    • Each list element corresponds to a run in the log, in order.
    • Every element is {juror: [[comment, stance], …]}.
    """
    lines = Path(path).read_text(encoding="utf-8", errors="ignore").splitlines()
    runs = _split_runs(lines)
    return [_parse_single_run(r) for r in runs if r]

# ―― run & inspect ――――――――――――――――――――――――――――――――――――
runs_data = parse_jury_log(file_path)
runs_data

[{'Michelle Chavez': [["Well, that defense lawyer sure did a number on Mrs. Cohen. If she couldn't even read the newspaper, how could she be sure she saw Tomer and Stan? It makes me question the whole case.",
    'UNDECIDED'],
   ["The eyewitness testimony is unreliable due to the witness's vision issues and recent lens replacement, creating reasonable doubt.",
    'NOT GUILTY']],
  'Rebecca Martin': [["Michelle makes a good point. Mrs. Cohen's testimony is definitely shaky now. The fact that she had just gotten new glasses and couldn't read the headline casts serious doubt on her ability to accurately identify the defendants.",
    'UNDECIDED'],
   ["The eyewitness testimony is unreliable due to the witness's vision issues and recent lens replacement, creating reasonable doubt.",
    'NOT GUILTY']]}]

In [ ]:
"""
metrics.py  –  Tiny helpers for jury‑simulation analysis
========================================================
Input format throughout:
    runs: List[RunDict]
    RunDict  = { "Juror name": [ [comment, stance], ... ] }

All stances are normalised to upper‑case strings.
"""

from collections import Counter
from typing import Dict, List, Optional

RunDict   = Dict[str, List[List[str]]]          # juror ➜ [[comment, stance], …]
RunsList  = List[RunDict]


# ─────────────────────────── Juror‑level ────────────────────────────
def stance_flip_count(sequence: List[List[str]]) -> int:
    """Return how many times the juror changed stance during a run."""
    flips = 0
    for (_, s1), (_, s2) in zip(sequence, sequence[1:]):
        if s1 and s2 and s1 != s2:
            flips += 1
    return flips


def first_commit_round(sequence: List[List[str]]) -> Optional[int]:
    """Index (0‑based) of first non‑UNDECIDED stance, or None if never commits."""
    for i, (_, stance) in enumerate(sequence):
        if stance and stance != "UNDECIDED":
            return i
    return None


def verbosity(sequence: List[List[str]]) -> float:
    """Average characters per comment for one juror in a run."""
    return sum(len(c) for c, _ in sequence) / len(sequence)


# ─────────────────────────── Run‑level ──────────────────────────────
def _round_stances(run: RunDict) -> List[List[str]]:
    """Helper: transposes run into [[s_juror1, s_juror2, …] per round]."""
    rounds = max(len(seq) for seq in run.values())
    jurors = list(run)
    out = []
    for r in range(rounds):
        out.append([
            (run[j][r][1] if r < len(run[j]) else "UNDECIDED").upper()
            for j in jurors
        ])
    return out


def consensus_speed(run: RunDict) -> Optional[int]:
    """
    Round index where all jurors share the same *non‑UNDECIDED* stance.
    Returns None if never reaches consensus before verdict block.
    """
    for idx, stances in enumerate(_round_stances(run)):
        uniq = {s for s in stances if s != "UNDECIDED"}
        if len(uniq) == 1 and len(uniq) == len(set(stances)):
            return idx
    return None


def majority_volatility(run: RunDict) -> int:
    """Count how many times the majority stance flips before verdict."""
    majority = []
    for stances in _round_stances(run):
        if all(s == "UNDECIDED" for s in stances):
            majority.append("UNDECIDED")
        else:
            mc = Counter([s for s in stances if s != "UNDECIDED"]).most_common(1)[0][0]
            majority.append(mc)
    return sum(m1 != m2 for m1, m2 in zip(majority, majority[1:]))


def agreement_ratio_per_round(run: RunDict) -> List[float]:
    """List of % jurors sharing the majority stance at each round."""
    ratios = []
    n = len(run)
    for stances in _round_stances(run):
        if all(s == "UNDECIDED" for s in stances):
            ratios.append(0.0)
        else:
            cnt = Counter([s for s in stances if s != "UNDECIDED"])
            ratios.append(max(cnt.values()) / n)
    return ratios


# ─────────────────────── Across‑runs aggregates ─────────────────────
def verdict_distribution(runs: RunsList) -> Dict[str, int]:
    """How often each final stance (‘GUILTY’, ‘NOT GUILTY’, ‘UNDECIDED’) wins."""
    tally = Counter()
    for run in runs:
        finals = [seq[-1][1].upper() for seq in run.values() if seq]
        if finals:                         # assume majority of finals decides case
            winner = Counter(finals).most_common(1)[0][0]
            tally[winner] += 1
    return dict(tally)


def avg_rounds_to_consensus(runs: RunsList) -> float:
    """Average consensus_speed across runs (ignores runs with None)."""
    speeds = [s for r in runs if (s := consensus_speed(r)) is not None]
    return sum(speeds) / len(speeds) if speeds else float("nan")


def juror_steadiness(runs: RunsList) -> Dict[str, float]:
    """Fraction of runs where each juror never flips stance."""
    counts = Counter()
    steady = Counter()
    for run in runs:
        for juror, seq in run.items():
            counts[juror] += 1
            if stance_flip_count(seq) == 0:
                steady[juror] += 1
    return {j: steady[j] / counts[j] for j in counts}

In [ ]:
for idx, run in enumerate(runs_data, 1):
    print(f"\n=== Run {idx} ===")
    for juror, seq in run.items():
        print(f"  {juror}")
        print(f"    flips: {stance_flip_count(seq)}")
        print(f"    first commit round: {first_commit_round(seq)}")
        print(f"    verbosity (chars/comment): {verbosity(seq):.1f}")
    print("  run‑level:")
    print(f"    consensus speed: {consensus_speed(run)}")
    print(f"    majority volatility: {majority_volatility(run)}")
    print(f"    agreement ratio per round: {agreement_ratio_per_round(run)}")

print("\n=== Across runs ===")
print("  verdict distribution:", verdict_distribution(runs_data))
print("  avg rounds to consensus:", avg_rounds_to_consensus(runs_data))
print("  juror steadiness:", juror_steadiness(runs_data))


=== Run 1 ===
  Michelle Chavez
    flips: 1
    first commit round: 1
    verbosity (chars/comment): 158.5
  Rebecca Martin
    flips: 1
    first commit round: 1
    verbosity (chars/comment): 196.5
  run‑level:
    consensus speed: 1
    majority volatility: 1
    agreement ratio per round: [0.0, 1.0]

=== Run 2 ===
  Michelle Chavez
    flips: 1
    first commit round: 1
    verbosity (chars/comment): 154.0
  Rebecca Martin
    flips: 1
    first commit round: 1
    verbosity (chars/comment): 189.0
  run‑level:
    consensus speed: 1
    majority volatility: 1
    agreement ratio per round: [0.0, 1.0]

=== Run 3 ===
  Michelle Chavez
    flips: 1
    first commit round: 1
    verbosity (chars/comment): 160.5
  Rebecca Martin
    flips: 1
    first commit round: 1
    verbosity (chars/comment): 178.0
  run‑level:
    consensus speed: 1
    majority volatility: 1
    agreement ratio per round: [0.0, 1.0]

=== Run 4 ===
  Michelle Chavez
    flips: 1
    first commit round: 1
    ver